# Markov Micromanufacturing

The Markov Micromanufacturing Company has 9 production jobs it needs to process in the next 24 hours. The company has 4 identical machines that run in parallel. Each of these 9 jobs must be run on one of these machines nonpreemptively: that is, once a job is started on a machine, it must stay on that machine until it is completed. The processing times of these jobs are given below:

| job                     | 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 |
|:------------------------|:--|:--|:--|:--|:--|:--|:--|:--|:--|
| processing time (hours) | 7 | 7 | 6 | 6 | 5 | 5 | 4 | 4 | 4 |

The company wants to minimize the makespan, or the completion time of the last job to finish processing.

We formulated this problem as a dynamic program in class. 

* Solve this dynamic program. 
  - What is the minimum makespan?
  - How should the jobs be assigned to machines to achieve this minimum makespan?

_Hint._ We showed in class that LPT achieves a makespan of 15, so we know that we can't do worse than that. So, in your code, only create states that correspond to machine loads between 0 and 15 to keep your dynamic program relatively small.

_Hint._ To get the maximum of a collection of numbers, you can use the `max()` function. For example:

```python
max(3, 0, -10, 5)
```

In [1]:
# Write your code here
# Import networkx and bellmanford
import networkx as nx
import bellmanford as bf

# Create variables for number of jobs and maximum load per machine
N_JOBS = 9
MAX_LOAD = 15

# Create list for processing times
time = [7, 7, 6, 6, 5, 5, 4, 4, 4]

# Create empty graph
G = nx.DiGraph()

# Add stage-state nodes (t, n1, n2, n3, n4)
#   t = stage, consider job t
#   n1 = load on machine 1
#   n2 = load on machine 2
#   n3 = load on machine 3
#   n4 = load on machine 4
for t in range(0, N_JOBS + 1):
    for n1 in range(0, MAX_LOAD + 1):
            for n2 in range(0, MAX_LOAD + 1):
                    for n3 in range(0, MAX_LOAD + 1):
                            for n4 in range(0, MAX_LOAD + 1):
                                G.add_node((t, n1, n2, n3, n4))

# Add end node
G.add_node("end")

# Add edges corresponding to adding job t on a machine
for t in range(0, N_JOBS):
    for n1 in range(0, MAX_LOAD + 1):
            for n2 in range(0, MAX_LOAD + 1):
                    for n3 in range(0, MAX_LOAD + 1):
                            for n4 in range(0, MAX_LOAD + 1):
                                
                                # Add job t to machine 1
                                G.add_edge((t, n1, n2, n3, n4), (t + 1, n1 + time[t], n2, n3, n4), 
                                           length=0)

                                # Add job t to machine 2
                                G.add_edge((t, n1, n2, n3, n4), (t + 1, n1, n2 + time[t], n3, n4), 
                                           length=0)


                                # Add job t to machine 3
                                G.add_edge((t, n1, n2, n3, n4), (t + 1, n1, n2, n3 + time[t], n4), 
                                           length=0)


                                # Add job t to machine 4
                                G.add_edge((t, n1, n2, n3, n4), (t + 1, n1, n2, n3, n4 + time[t]), 
                                           length=0)


# Add edges from last stage to end node
for n1 in range(0, MAX_LOAD + 1):
        for n2 in range(0, MAX_LOAD + 1):
                for n3 in range(0, MAX_LOAD + 1):
                        for n4 in range(0, MAX_LOAD + 1):
                            G.add_edge((N_JOBS, n1, n2, n3, n4), "end", length=max(n1, n2, n3, n4))

# Solve the DP/shortest path problem using the Bellman-Ford algorithm
length, nodes, negative_cycle = bf.bellman_ford(G, source=(0, 0, 0, 0, 0), target="end", 
                                                weight="length")

print("Negative cycle? {0}".format(negative_cycle))
print("Shortest path length: {0}".format(length))
print("Shortest path: {0}".format(nodes))

Negative cycle? False
Shortest path length: 12
Shortest path: [(0, 0, 0, 0, 0), (1, 7, 0, 0, 0), (2, 7, 7, 0, 0), (3, 7, 7, 6, 0), (4, 7, 7, 12, 0), (5, 12, 7, 12, 0), (6, 12, 12, 12, 0), (7, 12, 12, 12, 4), (8, 12, 12, 12, 8), (9, 12, 12, 12, 12), 'end']


<!-- Write your notes here. Double-click to edit. -->
* The minimum makespan is 12.

* To achieve this minimum makespan, we should put:
    - job 0 on machine 3,
    - job 1 on machine 2, 
    - job 2 on machine 1,
    - job 3 on machine 1, 
    - job 4 on machine 3,
    - job 5 on machine 2,
    - job 6 on machine 4,
    - job 7 on machine 4,
    - job 8 on machine 4.